# **Notebook Listados OC SME Engagement**
## Inicialización
1. Drive setup
2. Definición de funciones

---

## Listados de FS, ST y AP
1. Carga del output de prioridades (listado madre)
2. Limpieza de columnas (opcional)
3. Ejecución

---

## Listados de Beneficios (Rebate o FD)
1. Carga del output de beneficios
2. Ejecución

---

## Listados de Combos
1. Carga del output de combos
2. Ejecución

# Inicialización

## Drive setup

**1.** Importa todas las librerías necesarias, configura credenciales de Google (Colab, Sheets, Drive) y monta el Drive.

In [ ]:
from google.oauth2.service_account import Credentials
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import os
import numpy as np
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
import gspread
from datetime import datetime, timedelta
import calendar
from google.colab import auth
auth.authenticate_user()
from google.auth import default
creds, _ = default()
service = build('sheets', 'v4', credentials=creds)
client = gspread.authorize(creds)
from google.colab import drive
drive.mount("/content/drive", force_remount=True)
import ipywidgets as widgets
import gdown
#from pydrive.auth import GoogleAuth
#from pydrive.drive import GoogleDrive
from oauth2client.client import GoogleCredentials
from openpyxl import load_workbook
#gauth = GoogleAuth()
#gauth.credentials = GoogleCredentials.get_application_default()
#drive = GoogleDrive(gauth)
import math
from google.colab import files

Mounted at /content/drive


## Definición de funciones

In [ ]:

def upload(df, country, name, mes, dry_run):
    if dry_run == True:
      output_directory = f"/content/drive/MyDrive/Notebook Listados/Run Tests/" # DRY RUN (GUARDA LISTADOS EN LA CARPETA DE PRUEBAS)
    else:
      output_directory = "/content/drive/Shared drives/Analytics_RCentral/Promos Automation/Engagement/Events/XLSX/" # RUN REAL (GUARDA LISTADOS EN LA CARPETA DEL EVENTS)

    filename = f"{output_directory}{country}_{name}_{mes}.csv"

    columna2 = pd.concat([pd.Series([df['shop_id'].iloc[0]]), df['shop_id']], ignore_index=True)
    os.makedirs(os.path.dirname(filename), exist_ok=True)
    columna2.to_csv(filename, index=False, header=False)
    print(f"✅ {country}_{name}_{mes}, se invitaron ", len(df), " restaurantes al evento")



In [ ]:
def get_date(fecha=None):
    if fecha == -1:
        fecha = datetime.now()
    elif fecha is None:
        fecha = datetime.now()
        fecha = (fecha.replace(day=1) + timedelta(days=32)).replace(day=1)

    return fecha.strftime("%b%y")


# FS, ST, AP

## Seteo de inputs


Carga el CSV principal de trabajo, normaliza el shop_id, **HAY QUE CAMBIAR EL NOMBRE DEL LISTADO**

In [ ]:
# Carga de los listados globales (output de prioridades)

archivo_csv = '/content/drive/MyDrive/Notebook Listados/Listados OC Feb-Mar v2.csv' # PONER EL NOMBRE DEL LISTADO MADRE

df = pd.read_csv(archivo_csv, encoding='latin1', dtype={'shop_id':str})
#df['shop_id'] = df['shop_id'].astype('str')

#df["G2"] = df["G"].replace(["G2", "G3", "G4"], "G5")
df

/tmp/ipython-input-1252894626.py:5: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(archivo_csv, encoding='latin1', dtype={'shop_id':str})


,country_code,shop_id,shop_name,bdl,bdm,bd,ciudad,neighborhood,ba,area_name,...,Invitado a Flagship New Rs,Invitado a Flagship Core,Invitado a Flagship Mexicali,Invitado a Flagship Puebla,Invitado a Flagship Culiacan,Invitado a Flagship Leon,Invitado a AP Dummy,Invitado a postulaciÃ³n de FS,count_invitado,tiene_invitado
0,CO,5764608898523466301,dbarril,danielmontalvo,jakelinevelasquez,josuepallaresgalvan,MedellÃ­n,NaN,BA_1,BDArea 41,...,NaN,Invitado,NaN,NaN,NaN,NaN,Invitado,NaN,2,True
1,CO,5764608187593130681,La Cocina de Claudia,danielmontalvo,jakelinevelasquez,josuepallaresgalvan,MedellÃ­n,NaN,BA_1,BDArea 41,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Invitado,2,True
2,CO,5764608437493957082,Yum!Dino,NaN,NaN,NaN,MedellÃ­n,NaN,BA_1,BDArea 41,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Invitado,2,True
3,CO,5764607603234309234,Rellenit@s,NaN,NaN,NaN,MedellÃ­n,NaN,BA_1,BDArea 41,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Invitado,2,True
4,CO,5764607717537482352,Patacones Curazao,NaN,NaN,NaN,MedellÃ­n,NaN,BA_1,BDArea 41,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Invitado,2,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88062,PE,5764607731017975666,Don Gallardo,NaN,NaN,NaN,Lima,NaN,BA_2,BDArea 8 - 0 - D0 - Callao,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Invitado,2,True
88063,PE,5764607660599806927,La Ruta de las Hamburguesas,NaN,NaN,NaN,Lima,NaN,BA_2,BDArea 8 - 0 - D0 - Callao,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Invitado,2,True
88064,PE,5764607543067019212,Salchi Lovers,NaN,NaN,NaN,Lima,NaN,BA_2,BDArea 8 - 0 - D0 - Callao,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Invitado,2,True
88065,PE,5764607550650321298,Papi Riqui,NaN,NaN,NaN,Lima,NaN,BA_2,BDArea 8 - 0 - D0 - Callao,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Invitado,2,True


In [ ]:

# df["G2"].value_counts()

Limpiar el df de columnas innecesarias (eventos que no se cargaran este mes)

In [ ]:
df.columns

Index(['country_code', 'shop_id', 'shop_name', 'bdl', 'bdm', 'bd', 'ciudad',
       'neighborhood', 'ba', 'area_name', 'category', 'second_category',
       'primer_fecha_de_conexion', 'dias_sin_conectarse',
       'reviews_num_competencia', 'inicio', 'fin', 'daily_orders', 'asp',
       'aop', 'new_potential', 'city_thr', 'bda_thr', 'prioridad',
       'prioridad_ajustada_engagement', 'cambio_engagement',
       'prioridad_ajustada_sales_ops', 'cambio_sales_ops',
       'prioridad_a_cargar', '-', 'comentario',
       'comentario_ajustado_sales_ops', 'Invitado a Flagship New Rs',
       'Invitado a Flagship Core', 'Invitado a Flagship Mexicali',
       'Invitado a Flagship Puebla', 'Invitado a Flagship Culiacan',
       'Invitado a Flagship Leon', 'Invitado a AP Dummy',
       'Invitado a postulaciÃ³n de FS', 'count_invitado', 'tiene_invitado'],
      dtype='object')

In [ ]:
# Hacer Copy paste del output de la celda anterior y quitar las columnas que no sirven: cc, shop e invitaciones y grupo FS, OJO QUITAMOS MINUS TRANSICIÓN

#column_names = ['country_code', 'shop_id',
#             'Invitado a FS minus transiciÃ³n',
#             'Invitado a FS minus',
#             'Invitado a Flagship New Rs',
#             'Invitado a postulaciÃ³n de FS',
#             'Invitado a FS P4',
#             #'Invitado a Autopromo %Off',
#             'Invitado a Autopromo FD',
#             'MX Invitado a ST DÃ­a de Mexican',
#             'MX Invitado a ST DÃ­a de Asian & International',
#             'MX Invitado a ST DÃ­a de Fast Food',
#             'CO Invitado a ST DÃ­a de Colombiana y saludable',
#             'CO Invitado a ST DÃ­a de Asian & International',
#             'CO Invitado a ST DÃ­a de Fast Food',
#             'Invitado a Flagship New Users',
#             'Invitado a Flagship Excepciones',
#             'Invitado a Impulso de conversiÃ³n',
#             'G2'
#             #'Invitado CR dÃ­a de la Madre',
#             #'Invitado PE dÃ­a del niÃ±o'
#              ]
#
#
#df = df[column_names]
#
#df

**Diccionario de promociones**


nomenclaturas:

    'GP_NewRs',
    'FS_Acq',
    'FS_minus_transition',
    'FS_minus', 'Postulados',
    'AP_off',
    'AP_Delivery',
    'Diasde_Mexicana',
    'Diasde_Asiatica',
    'Diasde_Fast_food',
    'Diasde_Local_Healthy',
    'Diasde_Asiatica',
    'Diasde_Fast_food',
    'ST_wknd_All',
    'ST_wknd_FF',
    'Conversion',
    'TC',
    'R_one',
    'TC_post',
    'R_one_post'


columnas:

          'Invitado a GP NewRs',
          'Invitado a FS G1 Acq',
          'Invitado a FS minus transiciÃ³n',
          'Invitado a FS minus',
          'Invitado a postulaciÃ³n de FS',
          'Invitado a Autopromo %Off',
          'Invitado a Autopromo FD',
          'MX Invitado a ST DÃ­a de Mexican',
          'MX Invitado a ST DÃ­a de Asian & International',
          'MX Invitado a ST DÃ­a de Fast Food',
          'CO Invitado a ST DÃ­a de Colombiana y saludable',
          'CO Invitado a ST DÃ­a de Asian & International',
          'CO Invitado a ST DÃ­a de Fast Food',
          'Invitado ST Fines de semana y dÃ­as de semana less18DO',
          'Invitado ST Fines de semana y dÃ­as de semana plus18DO',
          'Invitado a impulso de conversiÃ³n',
          'Invitado a GP Top Competitor',
          'Invitado a GP R1',
          'Invitado a postulaciÃ³n de GP Top Competitor',
          'Invitado a postulaciÃ³n de GP R1'


## Construcción

In [ ]:

def generar_listados(df, eventos, countries, mes, dry_run=False):
    if dry_run==True:
      print("Ejecutando DRY RUN | Los listados se guardaran en la carpeta de pruebas\n")
    else:
      print("Generando listados | Los listados creados se guardaran en la carpeta del Events Automation\n")
    mx = []
    co = []
    pe = []
    cr = []
    CC = [mx, co, pe, cr]

    for evento in eventos:
        col = evento["columna"]
        tipo = evento["tipo"]
        nombre_evento = evento["nombre"]

        for c in countries:
            if tipo == "simple":
                filtrado = df[(df[col] == "Invitado") & (df['country_code'] == c)]
                if not filtrado.empty:
                    upload(filtrado, c, nombre_evento, mes, dry_run)
                    if c == 'MX': mx.append(f"{c}_{nombre_evento}_{mes}.csv")
                    if c == 'CO': co.append(f"{c}_{nombre_evento}_{mes}.csv")
                    if c == 'PE': pe.append(f"{c}_{nombre_evento}_{mes}.csv")
                    if c == 'CR': cr.append(f"{c}_{nombre_evento}_{mes}.csv")

            elif tipo == "g":
                valores_g = df["G2"].dropna().unique()
                for g in valores_g:
                    filtrado = df[(df['country_code'] == c) & (df[col] == "Invitado") & (df["G2"] == g)]
                    if not filtrado.empty:
                        upload(filtrado, c, f"{g}_Men", mes, dry_run)

    #for c in countries:
    #  filtrado = df[(df['Invitado a Flagship New Users'] == 'Invitado') & (df['country_code'] == c)]
    #  if not filtrado.empty:
    #    upload(filtrado,c,'AP_Dummy_G1_G5', mes, dry_run)
    #    if c == 'MX': mx.append(f"{c}_AP_Dummy_G1_G5_{mes}.csv")
    #    if c == 'CO': co.append(f"{c}_AP_Dummy_G1_G5_{mes}.csv")
    #    if c == 'PE': pe.append(f"{c}_AP_Dummy_G1_G5_{mes}.csv")
    #    if c == 'CR': cr.append(f"{c}_AP_Dummy_G1_G5_{mes}.csv")

    print("Listados generados correctamente!\n")
    for l in CC:
      for e in l:
        print(f"{e}")
    numEv = (len(mx)+len(co)+len(pe)+len(cr))
    print(f'\nHay {numEv} eventos este mes!')

## Ejecución

In [ ]:
df.columns

Index(['country_code', 'shop_id', 'shop_name', 'bdl', 'bdm', 'bd', 'ciudad',
       'neighborhood', 'ba', 'area_name', 'category', 'second_category',
       'primer_fecha_de_conexion', 'dias_sin_conectarse',
       'reviews_num_competencia', 'inicio', 'fin', 'daily_orders', 'asp',
       'aop', 'new_potential', 'city_thr', 'bda_thr', 'prioridad',
       'prioridad_ajustada_engagement', 'cambio_engagement',
       'prioridad_ajustada_sales_ops', 'cambio_sales_ops',
       'prioridad_a_cargar', '-', 'comentario',
       'comentario_ajustado_sales_ops', 'Invitado a Flagship New Rs',
       'Invitado a Flagship Core', 'Invitado a Flagship Mexicali',
       'Invitado a Flagship Puebla', 'Invitado a Flagship Culiacan',
       'Invitado a Flagship Leon', 'Invitado a AP Dummy',
       'Invitado a postulaciÃ³n de FS', 'count_invitado', 'tiene_invitado'],
      dtype='object')

In [ ]:
# Configuración de eventos

EVENTOS = [
    {"nombre": "FS_Acq", "columna": "Invitado a Flagship New Rs", "tipo": "simple"},
    {"nombre": "FS_Core", "columna": 'Invitado a Flagship Core', "tipo": "simple"},
    {"nombre": "FS_Mxcl", "columna": 'Invitado a Flagship Mexicali', "tipo": "simple"},
    {"nombre": "FS_Pueb", "columna": 'Invitado a Flagship Puebla', "tipo": "simple"},
    {"nombre": "FS_Culc", "columna": 'Invitado a Flagship Culiacan', "tipo": "simple"},
    {"nombre": "FS_Leon", "columna": 'Invitado a Flagship Leon', "tipo": "simple"},
    {"nombre": "AP_D", "columna": 'Invitado a AP Dummy', "tipo": "simple"},
    {"nombre": "FS_Postulados", "columna": 'Invitado a postulaciÃ³n de FS', "tipo": "simple"}


    #{"nombre": "", "columna": , "tipo": "simple"},
    #{"nombre": "Conversion", "columna": 'Invitado a Impulso de conversiÃ³n', "tipo": "simple"},
    #{"nombre": "AP_FD", "columna": 'ST AP FD', "tipo": "simple"},
    #{"nombre": "ST_50Off", "columna": 'Invitado a ST 50% Off', "tipo": "simple"},
    #{"nombre": "Seasonality_BlackFriday", "columna": 'Invitado a Black Friday', "tipo": "simple"},
    #{"nombre": "Seasonality_BuenFin", "columna": 'Invitado a Buen Fin', "tipo": "simple"}

    #{"nombre": "Diasde_Mexicana_NU", "columna": 'MX Invitado a ST DÃ­a de Mexican NewU', "tipo": "simple"},
    #{"nombre": "Diasde_Mexicana_Exc", "columna": 'MX Invitado a ST DÃ­a de Mexican Exc', "tipo": "simple"},
    #{"nombre": "Diasde_AsianInt_NU", "columna": 'MX Invitado a ST DÃ­a de Asian & International NewU', "tipo": "simple"},
    #{"nombre": "Diasde_AsianInt_Exc", "columna": 'MX Invitado a ST DÃ­a de Asian & International Exc', "tipo": "simple"},
    #{"nombre": "Diasde_Fast_food_NU", "columna": 'MX Invitado a ST DÃ­a de Fast Food NewU', "tipo": "simple"},
    #{"nombre": "Diasde_Fast_food_Exc", "columna": 'MX Invitado a ST DÃ­a de Fast Food Exc', "tipo": "simple"},
    #{"nombre": "Diasde_Local_Healthy_NU", "columna": 'CO Invitado a ST DÃ­a de Colombiana y saludable NewU', "tipo": "simple"},
    #{"nombre": "Diasde_Local_Healthy_Exc", "columna": 'CO Invitado a ST DÃ­a de Colombiana y saludable Exc', "tipo": "simple"},
    #{"nombre": "Diasde_AsianInt_NU", "columna": 'CO Invitado a ST DÃ­a de Asian & International NewU', "tipo": "simple"},
    #{"nombre": "Diasde_AsianInt_Exc", "columna": 'CO Invitado a ST DÃ­a de Asian & International Exc', "tipo": "simple"},
    #{"nombre": "Diasde_Fast_food_NU", "columna": 'CO Invitado a ST DÃ­a de Fast Food NewU', "tipo": "simple"},
    #{"nombre": "Diasde_Fast_food_Exc", "columna": 'CO Invitado a ST DÃ­a de Fast Food Exc', "tipo": "simple"},
    #{"nombre": "DiaMuertos99", "columna": 'Invitado Dia de muertos 99', "tipo": "simple"},
    #{"nombre": "DiaMuertos29", "columna": 'Invitado Dia de muertos 29', "tipo": "simple"},
    #{"nombre": "Halloween", "columna": 'Invitado halloween', "tipo": "simple"},
]

'Invitado a Flagship New Rs'





'Invitado a Flagship New Rs'

In [ ]:
print(get_date())


Feb26


In [ ]:
# Generación de listados

# Para guardar listados en la carpeta del events dry_run = False

generar_listados(df, EVENTOS, ["MX","CO","PE","CR"], 'Feb_Mar_26', dry_run=False)

Generando listados | Los listados creados se guardaran en la carpeta del Events Automation

✅ MX_FS_Acq_Feb_Mar_26, se invitaron  204  restaurantes al evento
✅ CO_FS_Acq_Feb_Mar_26, se invitaron  26  restaurantes al evento
✅ PE_FS_Acq_Feb_Mar_26, se invitaron  2  restaurantes al evento
✅ CR_FS_Acq_Feb_Mar_26, se invitaron  8  restaurantes al evento
✅ MX_FS_Core_Feb_Mar_26, se invitaron  8535  restaurantes al evento
✅ CO_FS_Core_Feb_Mar_26, se invitaron  1888  restaurantes al evento
✅ PE_FS_Core_Feb_Mar_26, se invitaron  315  restaurantes al evento
✅ CR_FS_Core_Feb_Mar_26, se invitaron  267  restaurantes al evento
✅ MX_FS_Mxcl_Feb_Mar_26, se invitaron  384  restaurantes al evento
✅ MX_FS_Pueb_Feb_Mar_26, se invitaron  349  restaurantes al evento
✅ MX_FS_Culc_Feb_Mar_26, se invitaron  218  restaurantes al evento
✅ MX_FS_Leon_Feb_Mar_26, se invitaron  213  restaurantes al evento
✅ MX_AP_D_Feb_Mar_26, se invitaron  9699  restaurantes al evento
✅ CO_AP_D_Feb_Mar_26, se invitaron  1888  rest

# Beneficios

## Seteo de inputs

In [ ]:
# Carga del listado de beneficios

beneficios_csv = '/content/drive/MyDrive/Notebook Listados/activarbeneficios.csv' # PONER EL NOMBRE DEL LISTADO MADRE

df = pd.read_csv(beneficios_csv, encoding='latin1', dtype={'shop_id':str})
df['fs_tiene_bxsy_activo'] = df['fs_tiene_bxsy_activo'].replace('-', np.nan)
df['fs_tiene_fd_activo'] = df['fs_tiene_fd_activo'].replace('-', np.nan)
#df['shop_id'] = df['shop_id'].astype('str')

#df["G2"] = df["G"].replace(["G2", "G3", "G4"], "G5")
df

,country_code,period,shop_id,Pack activado 01Dic,evento,registros_hasta,registrado_activo,tipo_cupos,bd,bdm,...,registrado_promo_espejo_recurrent,do,ranking_do,excedente,baja por duplicidad,act_ids,fs_tiene_bxsy_activo,fs_bxsy_fin,fs_tiene_fd_activo,fs_fd_fin
0,CO,2025_12,5764607604467435713,NaN,FS Normal,2025-12-07,1,jakelinevelasquez,josuepallaresgalvan,jakelinevelasquez,...,0,17.433333,6,0,NaN,[5765443701268481095],NaN,-,NaN,-
1,CO,2025_12,5764607667637848429,NaN,FS Normal,2025-12-07,1,fsolanillajaramillo,mariacamiladiaz,fsolanillajaramillo,...,0,4.133333,99,0,NaN,[5765439323455293149],NaN,-,NaN,-
2,CR,2025_12,5764607637644378836,NaN,FS Normal,2025-12-07,1,josemauriciomontero,jocelynaraya,josemauriciomontero,...,0,4.133333,17,0,NaN,[5765433904880092629],NaN,-,NaN,-
3,CR,2025_12,5764608605895265405,NaN,FS Normal,2025-12-07,1,josemauriciomontero,raulaviles,josemauriciomontero,...,0,9.666667,5,0,NaN,[5765436748429459887],NaN,-,1,2026-04-15
4,MX,2025_12,5764607523295068525,NaN,FS Normal,2025-12-07,1,Tijuana,senyasesanchezgarcia,monicariverayepiz,...,0,6.000000,88,0,NaN,[5765438426897648397],1,2025-12-28,1,2025-12-31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3371,MX,2026_01,5764607764949893274,NaN,FS Normal,2025-12-07,1,Saltillo,alozoyalatapi,rubensotogarza,...,0,17.666667,8,0,NaN,[5765436514383102528],NaN,-,NaN,-
3372,MX,2026_01,5764607872890306836,NaN,FS Normal,2025-12-07,1,Saltillo,alozoyalatapi,rubensotogarza,...,0,2.900000,30,0,NaN,[5765441864880556774],NaN,-,NaN,-
3373,MX,2026_01,5764608086191636964,NaN,FS Normal,2025-12-07,1,Saltillo,alozoyalatapi,rubensotogarza,...,0,12.033333,12,0,NaN,[5765441951211915763],NaN,-,NaN,-
3374,MX,2026_01,5764608185110105242,NaN,FS Normal,2025-12-07,1,Saltillo,alozoyalatapi,rubensotogarza,...,0,15.333333,10,0,NaN,[5765441969964648906],NaN,-,NaN,-


## Construcción

In [ ]:

def generar_beneficios(df, mes, dry_run=False):
    if dry_run==True:
      print("Ejecutando DRY RUN | Los listados se guardaran en la carpeta de pruebas\n")
    else:
      print("Generando listados | Los listados creados se guardaran en la carpeta del Events Automation\n")
    mx = []
    co = []
    pe = []
    cr = []
    CC = [mx, co, pe, cr]

    countries = ['MX', 'CO', 'PE']
    men_trim = {'FS Top Men': 'men','FS Trim Top New Users': 'trim'}
    col_sxsy = 'fs_tiene_bxsy_activo'
    col_fd = 'fs_tiene_fd_activo'
    col_trim_men = 'evento'
    brb = 'Beneficio_SxSy'
    bfd = 'Beneficio_FD'
    sinbeneficio = len(df[(df[col_fd]=='1') & (df[col_sxsy]=='1')])


    for c in countries:
      for e, n in men_trim.items():
            # RS sin rebate → subir rebate
            filtradorb = df[(df[col_sxsy].isna()) &
                        (df['country_code'] == c) &
                        (df[col_trim_men] == e)]
            if not filtradorb.empty:
              upload(filtradorb, c, f'{brb}_{n}', mes, dry_run)
              if c == 'MX': mx.append(f"{c}_{brb}_{n}_{mes}.csv")
              if c == 'CO': co.append(f"{c}_{brb}_{n}_{mes}.csv")
              if c == 'PE': pe.append(f"{c}_{brb}_{n}_{mes}.csv")

            # RS sin free delivery, excluyendo los que ya tienen rebate
            filtradofd = df[(df[col_fd].isna()) &
                        (df['country_code'] == c) &
                        (df[col_trim_men] == e)]
            filtradofd = filtradofd[~filtradofd.index.isin(filtradorb.index)]
            if not filtradofd.empty:
              upload(filtradofd, c, f'{bfd}_{n}', mes, dry_run)
              if c == 'MX': mx.append(f"{c}_{bfd}_{n}_{mes}.csv")
              if c == 'CO': co.append(f"{c}_{bfd}_{n}_{mes}.csv")
              if c == 'PE': pe.append(f"{c}_{bfd}_{n}_{mes}.csv")
# CR
    for e, n in men_trim.items():
      filtradofd = df[(df[col_fd].isna()) &
                      (df['country_code'] == 'CR') &
                      (df[col_trim_men] == e)]
      if not filtradofd.empty:
        upload(filtradofd, 'CR', f'{bfd}_{n}', mes, dry_run)
        cr.append(f"CR_{bfd}_{n}_{mes}.csv")

      filtradorb = df[(df[col_sxsy].isna()) &
                      (df['country_code'] == 'CR') &
                      (df[col_trim_men] == e)]
      filtradorb = filtradorb[~filtradorb.index.isin(filtradofd.index)]
      if not filtradorb.empty:
        upload(filtradorb, 'CR', f'{brb}_{n}', mes, dry_run)
        cr.append(f"CR_{brb}_{n}_{mes}.csv")


    print(f'❌ {sinbeneficio} Rs sin beneficio :(')



    print("Listados de beneficios generados correctamente!\n")
    eventos = ['SxSy', 'FD']   # prioridad de eventos
    paises = ['MX', 'CO', 'PE', 'CR']
    tipos = ['men', 'trim']

    for evento in eventos:          # primero tipo de evento
        for pais in paises:         # luego país
            lista = {'MX': mx, 'CO': co, 'PE': pe, 'CR': cr}[pais]
            for tipo in tipos:      # finalmente men o trim
                for archivo in lista:
                    if f"{pais}_Beneficio_{evento}_{tipo}_" in archivo:
                        print(archivo)



In [ ]:
def generar_beneficios_v2(df, mes, dry_run=False):
    if dry_run:
        print("Ejecutando DRY RUN | Los listados se guardarán en la carpeta de pruebas\n")
    else:
        print("Generando listados | Los listados creados se guardarán en Events Automation\n")

    paises = ["MX", "CO", "PE", "CR"]
    beneficios = {
        "rb": "Beneficio_SxSy",
        "fd": "Beneficio_FD"
    }

    col_sxsy = 'fs_tiene_bxsy_activo'
    col_fd   = 'fs_tiene_fd_activo'
    col_periodo = 'internal_act_name'

    # Detectar periodo (men / bim)
    df["periodo"] = df[col_periodo].str.contains("bimestral", case=False, na=False)\
                     .map({True: "bim", False: "men"})

    archivos_generados = {p: [] for p in paises}

    # RS sin ningún beneficio
    sinbeneficio = len(df[(df[col_fd] == '1') & (df[col_sxsy] == '1')])

    # --------- PROCESAMIENTO ---------
    for pais in paises:

        for periodo in ["men", "bim"]:
            df_base = df[
                (df['country_code'] == pais) &
                (df["periodo"] == periodo)
            ]

            if df_base.empty:
                continue

            # Prioridad según país
            if pais == "CR":
                orden_beneficios = ["fd", "rb"]  # CR prioriza FD → SxSy
            else:
                orden_beneficios = ["rb", "fd"]  # MX, CO, PE priorizan SxSy → FD

            usados = set()  # para excluir enchapes entre rb y fd

            # Procesar según prioridad asignada
            for tipo_ben in orden_beneficios:

                if tipo_ben == "rb":
                    filtrado = df_base[df_base[col_sxsy].isna()]
                else:
                    filtrado = df_base[df_base[col_fd].isna()]

                # excluir los que ya estuvieron en el beneficio previo
                filtrado = filtrado[~filtrado.index.isin(usados)]

                if filtrado.empty:
                    continue

                nombre = f"{beneficios[tipo_ben]}_{periodo}"
                upload(filtrado, pais, nombre, mes, dry_run)
                archivos_generados[pais].append(nombre)

                usados |= set(filtrado.index)

    print(f"❌ {sinbeneficio} RS sin beneficio :(")
    print("\nListados generados correctamente!\n")

    for pais in paises:
        print(f"\n====== Archivos {pais} ======")
        for archivo in archivos_generados[pais]:
            print(f'{pais}_{archivo}_{mes}.csv')


## Ejecución

In [ ]:
get_date()

'Jan26'

In [ ]:
generar_beneficios_v2(df, 'Dec25_2', dry_run=False)

Generando listados | Los listados creados se guardarán en Events Automation

✅ MX_Beneficio_SxSy_men_Dec25_2, se invitaron  1389  restaurantes al evento
✅ MX_Beneficio_SxSy_bim_Dec25_2, se invitaron  619  restaurantes al evento
✅ MX_Beneficio_FD_bim_Dec25_2, se invitaron  298  restaurantes al evento
✅ CO_Beneficio_SxSy_men_Dec25_2, se invitaron  729  restaurantes al evento
✅ CO_Beneficio_SxSy_bim_Dec25_2, se invitaron  14  restaurantes al evento
✅ CR_Beneficio_FD_men_Dec25_2, se invitaron  109  restaurantes al evento
✅ CR_Beneficio_FD_bim_Dec25_2, se invitaron  13  restaurantes al evento
✅ CR_Beneficio_SxSy_bim_Dec25_2, se invitaron  8  restaurantes al evento
❌ 197 RS sin beneficio :(

Listados generados correctamente!


====== Archivos MX ======
MX_Beneficio_SxSy_men_Dec25_2.csv
MX_Beneficio_SxSy_bim_Dec25_2.csv
MX_Beneficio_FD_bim_Dec25_2.csv

====== Archivos CO ======
CO_Beneficio_SxSy_men_Dec25_2.csv
CO_Beneficio_SxSy_bim_Dec25_2.csv

====== Archivos PE ======

====== Archivos CR =

# Combos

## Seteo de inputs


In [ ]:
# Carga del listado de combos

listado = '/content/drive/MyDrive/Notebook Listados/ListadoCombos0105.xlsx' # PONER EL NOMBRE DEL LISTADO MADRE

df = pd.read_excel(listado, dtype={'shop_id':str, 'item_id': str}, parse_dates=['create_date'], usecols = ['country_code', 'shop_id', 'shop_name', 'create_date', 'item_id', 'item_name', 'disc_perc', 'original_price', 'current_price'])
#df['shop_id'] = df['shop_id'].astype('str')
df

/usr/local/lib/python3.12/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,country_code,shop_id,shop_name,create_date,item_id,item_name,disc_perc,original_price,current_price
0,MX,5764607523068578918,Sushi koi Ejercito,2025-10-24,5764707550947901478,Combo Godín Boneless,15.0,249.0,211.0
1,MX,5764607523068578918,Sushi koi Ejercito,2025-10-24,5764707541657518123,Combo Alitas,10.0,225.0,202.5
2,MX,5764607523068578918,Sushi koi Ejercito,2025-05-07,5764703190142618223,Combo Godín Rollo+Bebida,59.0,245.0,99.0
3,MX,5764607523068578918,Sushi koi Ejercito,2023-09-23,5764695375298629130,4 X 3 Rollos Empanizados,30.0,735.0,514.5
4,MX,5764607523068578918,Sushi koi Ejercito,2023-09-23,5764695305119534590,4 X 3 Rollos Tradicionales,4.0,490.0,470.0
...,...,...,...,...,...,...,...,...,...
52611,MX,5764609611324458328,Snack del Paraíso,2025-09-20,5764706354724012127,Combo Paraiso Crepa,30.0,190.0,133.0
52612,CO,5764609679704196321,Madame Mazapán Repostería,2026-01-05,5764709713937894267,Antojito de Sal con Bebida,15.0,19700.0,16745.0
52613,MX,5764609808272195671,La porcina Carnitas,2025-10-31,5764707420790262253,Cemita Surtida y Coca 600ml,5.0,74.0,70.3
52614,MX,5764609808272195671,La porcina Carnitas,2025-10-31,5764707505200630207,Kilo de Carne Surtida y 2 Cocas de 600ml,20.0,468.0,374.4


## Construcción

In [ ]:

# Funciones para generar listados

def getNewCombosDateRange(n=0):
    # n = 0 → semana pasada
    # n = -1 → hace dos semanas
    # n = -2 → hace tres semanas, etc.

    hoy = datetime.today().replace(hour=0, minute=0, second=0, microsecond=0)

    # lunes de la semana pasada es hoy - (weekday+7)
    # si n < 0 movemos semanas adicionales: n * 7 días
    lunes = hoy - timedelta(days=hoy.weekday() + 7 + (-n * 7))
    domingo = lunes + timedelta(days=6)

    print(f'Lunes: {lunes.date()}, Domingo: {domingo.date()}')
    return lunes, domingo

def generar_OldCombos(df, oto, countriesTps, mes, dry_run=False):
    if dry_run==True:
      print("Ejecutando DRY RUN | Los listados se guardaran en la carpeta de pruebas\n")
    else:
      print("Generando listados | Los listados creados se guardaran en la carpeta del Events Automation\n")
    mx = []
    co = []
    pe = []
    cr = []
    CC = [mx, co, pe, cr]

    for c, tp in countriesTps.items():
            filtrado = df[(df['country_code'] == c) & (df['disc_perc'].between(20,50)) &
    (df['original_price'] * (1 - (df['disc_perc'] + oto) / 100) <= tp)] # --cambio 04/01/2026

            if not filtrado.empty:
                filtrado = filtrado[['shop_id']].drop_duplicates().reset_index(drop=True)
                upload(filtrado, c, 'OldCB', mes, dry_run)
                if c == 'MX': mx.append(f"{c}_OldCB_{mes}.csv")
                if c == 'CO': co.append(f"{c}_OldCB_{mes}.csv")
                if c == 'PE': pe.append(f"{c}_OldCB_{mes}.csv")
                if c == 'CR': cr.append(f"{c}_OldCB_{mes}.csv")


    print("Listados de Old Combos generados correctamente!\n")
    for l in CC:
      for e in l:
        print(f"{e}")
    numEv = (len(mx)+len(co)+len(pe)+len(cr))
    print(f'\nHay {numEv} eventos este mes!')

def generar_NewCombos(df, oto, countriesTps, mes, newCombosMon, newCombosSun, dry_run=False, download=False):
    if dry_run==True:
      print("Ejecutando DRY RUN | Los listados se guardaran en la carpeta de pruebas\n")
    else:
      print("Generando listados | Los listados creados se guardaran en la carpeta del Events Automation\n")
    mx = []
    co = []
    pe = []
    cr = []
    CC = [mx, co, pe, cr]

    for c, tp in countriesTps.items():
            filtrado = df[
                (df['country_code'] == c) &
                (df['disc_perc'].between(20, 50)) &
                (df['create_date'] >= newCombosMon) &
                (df['create_date'] <= newCombosSun) &
                (df['original_price'] * (1 - (df['disc_perc'] + oto) / 100) <= tp) # --cambio 04/01/2026
            ]
            if not filtrado.empty:
                filtrado = filtrado[['shop_id']].drop_duplicates().reset_index(drop=True)
                upload(filtrado, c, 'NewCB', mes, dry_run)
                if c == 'MX': mx.append(f"{c}_NewCB_{mes}.csv")
                if c == 'CO': co.append(f"{c}_NewCB_{mes}.csv")
                if c == 'PE': pe.append(f"{c}_NewCB_{mes}.csv")
                if c == 'CR': cr.append(f"{c}_NewCB_{mes}.csv")


    print("Listados de New Combos generados correctamente!\n")
    for l in CC:
      for e in l:
        print(f"{e}")
    numEv = (len(mx)+len(co)+len(pe)+len(cr))
    print(f'\nHay {numEv} eventos este mes!')

In [ ]:
# Funciones para generar plantillas

def getNewCombosDateRange(n=0):
    # n = 0 → semana pasada
    # n = -1 → hace dos semanas
    # n = -2 → hace tres semanas, etc.

    hoy = datetime.today().replace(hour=0, minute=0, second=0, microsecond=0)

    # lunes de la semana pasada es hoy - (weekday+7)
    # si n < 0 movemos semanas adicionales: n * 7 días
    lunes = hoy - timedelta(days=hoy.weekday() + 7 + (-n * 7))
    domingo = lunes + timedelta(days=6)

    print(f'Lunes: {lunes.date()}, Domingo: {domingo.date()}')
    return lunes, domingo


In [ ]:
def getTemplateName(cc, type, date = None):
  if date == None:
    date = datetime.today()
    dia = f'{date.day:02d}'
    mes = f'{date.month:02d}'
    anio = f'{date.year:02d}'
    dma = f'{anio}{mes}{dia}'
  else:
    dma = date

  name = f'{dma}_Reg{type}_{cc}'
  return name

In [ ]:
def getNewCombosTemplates(df, countriesTps, oto, mon, sun, date=None, dryRun=True):
  if dryRun:
    output_directory = f"/content/drive/MyDrive/Notebook Listados/Run Tests/" # DRY RUN (GUARDA LISTADOS EN LA CARPETA DE PRUEBAS)
    print(f'Los templates se guardaran en la carpeta de pruebas')
  else:
    output_directory = "/content/drive/Shared drives/Analytics_RCentral/Promos Automation/Engagement/Events/Templates/" # RUN REAL (GUARDA LISTADOS EN LA CARPETA DEL EVENTS)
    print(f'Los templates se guardaran en el Events Automation')

  invLists = {'MX': [], 'CO': [], 'PE': [], 'CR': []}

  newCombosMon, newCombosSun = mon, sun

  for c, tp in countriesTps.items():
    newCombos = df[(df['country_code'] == c) & (df['disc_perc'].between(20,50)) &
    (df['create_date'] >= newCombosMon ) & (df['create_date'] <= newCombosSun) &
    (df['original_price'] * (1 - (df['disc_perc'] + oto) / 100) <= tp)] # --cambio 04/01/2026
    newCombos = newCombos[['shop_id', 'item_id']]
    newCombos['OTO'] = oto
    templateName = getTemplateName(c, 'NewCB', date)
    newShops = newCombos['shop_id'].unique()
    if not newCombos.empty:
      print(f'✅ {templateName} se registraran {len(newCombos)} items en {len(newShops)} Rs para NewCombos')
      createCombosTemplate(newCombos, templateName, output_directory)
      invLists[c] = newCombos['shop_id'].unique().tolist()

  for country, lst in invLists.items():
    print(country)
    for rs in lst:
      print(rs)


In [ ]:
def getOldCombosTemplates(df, countriesTps, oto, date=None, dryRun=True):
  if dryRun:
    output_directory = f"/content/drive/MyDrive/Notebook Listados/Run Tests/" # DRY RUN (GUARDA LISTADOS EN LA CARPETA DE PRUEBAS)
    print(f'Los templates se guardaran en la carpeta de pruebas')
  else:
    output_directory = "/content/drive/Shared drives/Analytics_RCentral/Promos Automation/Engagement/Events/Templates/" # RUN REAL (GUARDA LISTADOS EN LA CARPETA DEL EVENTS)
    print(f'Los templates se guardaran en el Events Automation')

  invLists = {'MX': [], 'CO': [], 'PE': [], 'CR': []}

  for c, tp in countriesTps.items():
    oldCombos = df[(df['country_code'] == c) & (df['disc_perc'].between(20,50)) &
    (df['original_price'] * (1 - (df['disc_perc'] + oto) / 100) <= tp)] # --cambio 04/01/2026
    oldCombos = oldCombos[['shop_id', 'item_id']]
    oldCombos['OTO'] = oto
    templateName = getTemplateName(c, 'OldCB', date)
    shops = oldCombos['shop_id'].unique()
    print(f'✅ {templateName} se registraran {len(oldCombos)} items en {len(shops)} Rs para OldCombos')
    createCombosTemplate(oldCombos, templateName, output_directory)

In [ ]:
def createCombosTemplate(content, name, directory):
  template = load_workbook('/content/drive/MyDrive/Notebook Listados/combostemplate.xlsx')
  templateSheet = template['Worksheet']

  data = content.values.tolist()

  startRow = 2
  startCol = 1

  for i, fila in enumerate(data, start=startRow):
    for j, valor in enumerate(fila, start=startCol):
      templateSheet.cell(row=i, column=j, value=valor)

  full_path = f'{directory}{name}.xlsx'
  template.save(full_path)
  #files.download(full_path)  # <---- para descargarlo local


In [ ]:
# Funciones para actualizar lista de New Combos

def retrieveList(name, path = "/content/drive/MyDrive/Notebook Listados/Run Tests/"):
  filename = f"{name}.csv" if not name.endswith('.csv') else name
  listado = f"{path}{filename}"
  try:
      return pd.read_csv(listado, header=None, dtype=str)[0]
  except (FileNotFoundError, pd.errors.EmptyDataError):
      return pd.Series(dtype=str, name=0) # serie vacía pero con nombre 0 para que coincida en el concat


def updateNewCombosList(df, countriesTps, date, monday, sunday, oto):
  for c, tp in countriesTps.items():
    listName = f'{c}_NewCB_{date}'
    retrievedList = retrieveList(listName)
    retrievedRs = len(retrievedList)

    newCombos = df[(df['country_code'] == c) & (df['disc_perc'].between(20,50)) &
    (df['create_date'] >= monday ) & (df['create_date'] <= sunday) &
    (df['original_price'] * (1 - (df['disc_perc'] + oto).clip(upper=100) / 100) <= tp)]

    if not newCombos.empty:
      newShops = pd.Series(newCombos['shop_id'].unique(), dtype=str, name=0)
      updated_list = pd.concat([retrievedList, newShops], ignore_index=True).drop_duplicates()

      newRs = len(updated_list) - retrievedRs
      print(f'✅ {listName} se registraran {newRs} nuevos Rs para NewCombos de {date} (Antes: {retrievedRs} | Ahora {len(updated_list)})')

      output_path = f"/content/drive/MyDrive/Notebook Listados/Run Tests/{listName}.csv"
      updated_list.to_csv(output_path, header=False, index=False)

      files.download(output_path)
    else:
      print(f'❌ {listName} sin nuevos Rs para NewCombos de {date}')



## Ejecución


In [ ]:
get_date(-1)

'Jan26'

In [ ]:
countriesTps = {'MX':139, 'CO':22500, 'PE':35, 'CR':6000}

### New Combos

#### Listados

In [ ]:
mon, sun = getNewCombosDateRange()
otoNew = 10
generar_NewCombos(df, otoNew, countriesTps, get_date(-1), mon, sun, dry_run=True, download=False)

Ejecutando DRY RUN | Los listados se guardaran en la carpeta de pruebas

✅ MX_NewCB_Jan26, se invitaron  19  restaurantes al evento
✅ CO_NewCB_Jan26, se invitaron  4  restaurantes al evento
Listados de New Combos generados correctamente!

MX_NewCB_Jan26.csv
CO_NewCB_Jan26.csv

Hay 2 eventos este mes!


#### Actualizar listados

In [ ]:
mon, sun = getNewCombosDateRange()
updateNewCombosList(df, countriesTps, get_date(-1), mon, sun, otoNew)

Lunes: 2025-12-29, Domingo: 2026-01-04
✅ MX_NewCB_Jan26 se registraran 0 nuevos Rs para NewCombos de Jan26 (Antes: 53 | Ahora 53)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ CO_NewCB_Jan26 se registraran 0 nuevos Rs para NewCombos de Jan26 (Antes: 10 | Ahora 10)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

❌ PE_NewCB_Jan26 sin nuevos Rs para NewCombos de Jan26
✅ CR_NewCB_Jan26 se registraran 0 nuevos Rs para NewCombos de Jan26 (Antes: 1 | Ahora 1)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#### Plantillas

In [ ]:
getNewCombosDateRange()

Lunes: 2025-12-29, Domingo: 2026-01-04


(datetime.datetime(2025, 12, 29, 0, 0), datetime.datetime(2026, 1, 4, 0, 0))

In [ ]:
mon, sun = getNewCombosDateRange()
getNewCombosTemplates(df, countriesTps, otoNew, mon, sun, date=None,dryRun=False)

Lunes: 2025-12-29, Domingo: 2026-01-04
Los templates se guardaran en el Events Automation
✅ 20260105_RegNewCB_MX se registraran 49 items en 34 Rs para NewCombos
✅ 20260105_RegNewCB_CO se registraran 6 items en 6 Rs para NewCombos
✅ 20260105_RegNewCB_CR se registraran 1 items en 1 Rs para NewCombos
MX
5764607775934779348
5764608219973160229
5764608584252654556
5764609418873016286
5764608286738091343
5764607634146331799
5764608216877761503
5764607573865795533
5764607948563942969
5764608605194815310
5764607528751858643
5764608158727933544
5764608365695862164
5764609333003029808
5764608867879878690
5764607551497568876
5764608141220908455
5764608476475818207
5764607532233130033
5764607762366202804
5764609074772312390
5764608121788697880
5764607533210403473
5764607575019225723
5764607751142244992
5764607666295670624
5764607706376438072
5764607717847859594
5764607761846108552
5764607789192970640
5764608347643578641
5764608940642665253
5764609181907422280
5764609808272195671
CO
576460958774827

### Old Combos

#### Listados

In [ ]:
otoOld = 3
generar_OldCombos(df, otoOld, countriesTps, get_date(), dry_run=False)

Generando listados | Los listados creados se guardaran en la carpeta del Events Automation

✅ MX_OldCB_Jan26, se invitaron  4360  restaurantes al evento
✅ CO_OldCB_Jan26, se invitaron  1221  restaurantes al evento
✅ PE_OldCB_Jan26, se invitaron  292  restaurantes al evento
✅ CR_OldCB_Jan26, se invitaron  202  restaurantes al evento
Listados de Old Combos generados correctamente!

MX_OldCB_Jan26.csv
CO_OldCB_Jan26.csv
PE_OldCB_Jan26.csv
CR_OldCB_Jan26.csv

Hay 4 eventos este mes!


#### Templates

In [ ]:
otoOld = 3

getOldCombosTemplates(df, countriesTps, otoOld, dryRun=False)

Los templates se guardaran en el Events Automation
✅ 20251230_RegOldCB_MX se registraran 10818 items en 4360 Rs para OldCombos
✅ 20251230_RegOldCB_CO se registraran 3950 items en 1221 Rs para OldCombos
✅ 20251230_RegOldCB_PE se registraran 1222 items en 292 Rs para OldCombos
✅ 20251230_RegOldCB_CR se registraran 588 items en 202 Rs para OldCombos
